# Development history of a neighborhood in Oakland

I was curious how my neighborhood in Oakland, Cleveland Heights, was developed: which house was built first?  Which last?  How did this pattern fit with how the city was developed.  In particular, how much of that history can be read now.

## The Database

To find that data for this, I needed to know when each house was built, where each house was, and a list of houses.  Thanks in large part to the great collection of Oakland data at XXURLXX.  Initially, I found a GeoJSON file for containing parcel info for the county of Alameda, which includes Oakland and extends from Berkeley to Fremont north-to-south, and from the Bay to beyond Livermore east-to-west.  This was a huge file, and couldn't be processed in memory.  Several days later, I found parcel info for the City of Oakland from 20XX that conveniently fit into memory.

In [15]:
# insert code for reading shape file, print an entry
import fiona
import pprint

baseFile = 'data/Oakland_parcels/parcels'
source = fiona.open(baseFile + '.shp')
print('Number of parcels: %d\n' % len(source))
print('Second parcel:')
source.next()    # first entry has a lot of coordinates, so skip
pprint.pprint(source.next())

source.close()


Number of parcels 105351

Second parcel:
{'geometry': {'coordinates': [[(-122.244024425766, 37.86867162821933),
                               (-122.24398667467038, 37.86866162294282),
                               (-122.24399852865695, 37.868349112878676),
                               (-122.24401355395213, 37.86835277617628),
                               (-122.24404457097216, 37.868359870908144),
                               (-122.2440757393451, 37.86836653553029),
                               (-122.24408444864746, 37.86836826921976),
                               (-122.24407290434733, 37.86867278199862),
                               (-122.244024425766, 37.86867162821933)]],
              'type': 'Polygon'},
 'id': '1',
 'properties': OrderedDict([('OBJECTID', 23),
                            ('ADDR_HN', None),
                            ('ADDR_PD', None),
                            ('ADDR_SN', 'DWIGHT'),
                            ('ADDR_ST', 'WAY'),
                  

Fortunately, each entry also includes fields for the street address, though this information is missing here.  I'll catch these errors below.  If the street address wasn't present, it another call to reverse geocode the location (from Google, for instance) would have been required.  Note that there are 105,000 entries in all.

The construction history exists on Zillow, so this database was completed by repeat API calls.  The free Zillow API key that I have only allows 1000 queries per day.  Because I'm mostly interested in one neighborhood, I can approach this rate limit in a smart way by working outward from a central point.  I'll chooose the center location to be the Armenian Church on the top of the hill at McKinley and Spruce, since it's a big landmark close that's pretty central anyways.  (Another choice would have been the elmenetary school 3 blocks away.)  It will take 105 days (~3.5 months) to process all the data for Oakland.

To do this radial processing, the shapefile array was rewritten as a dictionary with the key being the distance from the landmark.  The distance was computed from the centroid of the parcel shape.  Technically, computing the centroid is unnecessary for radial proessing - the first coordinate of the parcel shape would suffice - but this may be useful later on.  Also, it doesn't take much time on such a small dataset.

In [20]:
import fiona
import geopy.distance
distance = geopy.distance.vincenty    
import shapely.geometry as shp
import pickle 

baseFile = 'data/Oakland_parcels/parcels'
center = (37.8058428, -122.2399758)        # (lat, long), Armenian Church

data_raw = {}
data_duplicates = {}

with fiona.drivers():           # Register format drivers with a context manager

    with fiona.open(baseFile + '.shp') as source:
       
        for f in source :
            if 'geometry' not in f:
                print('No geometry key in entry {}'.format(f))
            c = shp.shape(f['geometry']).centroid
            p = (c.y, c.x)

            d = round(distance(p, center).m * 10**6)/10**6        # round to micrometers
            f['centroid'] = p            

            if d in data_raw :
                if d in data_duplicates :   
                    data_duplicates[d].append(f)      # add to list in existing dictionary key
                else :
                    data_duplicates[d] = [data_raw[d], f]      # create list in existing dictionary key
            else :
                data_raw[d] = f
print('Number of parcels in dict: %d\n' % len(data_raw))

Number of parcels in dict: 97718



Notice that there are now about 97000 entries, about 8000 less than the original file.  It turns out that these are duplicate entries, such as condominiums, that share the same street address and coordinates but have different assessor parcel numbers (APNs).  Interestingly, the micrometer precision is necessary to distinguish parcels: rounding only to millimeters results in clashes between parcels that are distinct.  Given the number of parcels at a certain radius (when large), this isn't terribly surprising.

In [21]:
duplicateCount = 0
for (key, value) in data_duplicates.items() :
    duplicateCount += len(value)
print('Total number of duplicate parcels: %d' % (duplicateCount-len(data_duplicates)))

Total number of duplicate parcels: 7633


Since 7633+97718 = 105351, this accounts for the entire difference.  Now, save this dictionary to file.

In [ ]:
with open('data/OaklandParcels_inProcess.pkl', 'wb') as datafile :
    a = pickle.Pickler(datafile)
    compressed = {}
    compressed['data_raw'] = data_raw
    compressed['data_queried'] = {}
    compressed['data_errors'] = []
    a.dump(compressed)

### Adding year built from Zillow

To allow the processing to work piece-wise because of the 1000 queries/day limit, the input dictionary is processed starting with the smallest key.  After each entry is sent to the zillow API, it is popped from the input dictionary and placed in the output dictionary if the response is valid.  If the response is invalid, it is placed into an error dictionary for later processing.  There's also a rate limit of 10 queries/second, so a timer around the loop slows it down to that rate.

In [ ]:
import requests
import xmltodict
import time
import pickle

# Zillow variables keys
with open('../private/API_keys.pkl', 'rb') as datafile:
    zid = pickle.load(datafile)             # API key
zurl = 'http://www.zillow.com/webservice/GetDeepSearchResults.htm?'

inProcessFile = 'data/OaklandParcels_inProcess.pkl'
radius = 1000            # only process parcels within this radius - used for debugging
numToProcess = 1000      # zillow API limits to 1000 queries per day

# load data structures
with open(inProcessFile, 'rb') as fid :
    compressed = pickle.load(fid)
# ...and unpack
data_raw = compressed['data_raw']
data_queried = compressed['data_queried']
data_errors = compressed['data_errors']
del compressed

# sort keys by distance from closest to furthest
sortedKeys = [k for k in sorted(data_raw) if k < radius]

for (i, key) in zip(range(numToProcess), sortedKeys) :
    startT = time.time()            # set up timer to keep requests under 10/s

    try :
        # read in address details from input dictionary
        zp = {'address' : '{} {} {}'.format(data_raw[key]['properties']['ADDR_HN'],
                      data_raw[key]['properties']['ADDR_SN'],
                      data_raw[key]['properties']['ADDR_ST']),
              'citystatezip' : 'Oakland, CA ' + str(data_raw[key]['properties']['ZIP']),
              'zws-id' : zid}
        r = requests.get(zurl, params=zp)
        r_dict = xmltodict.parse(r.text)['SearchResults:searchresults']

        if r_dict['message']['code'] == '0' :       # valid response?
            r_dict = r_dict['response']['results']['result']
            
            # in case the response is a list of multiple (similar) entries, take the first one
            if type(r_dict)==list :
                r_dict = r_dict[0]
            # prune extraneous fields
            r_dict.pop('links')
            r_dict.pop('zestimate')
            r_dict.pop('localRealEstate')
            data_raw[key]['zillow'] = r_dict
            # transfer  to output dictionary
            data_queried[key] = data_raw.pop(key)
            
        else :
            print('For request {}, zillow code is {}. Here''s the record:'.format(
                    zp['address'], r_dict['message']['code']))
            print(r_dict)
            print('-'*60)
            # transfer info to error dictionary for offline analysis
            data_errors.append({'key': key, 'value': data_raw.pop(key), 
                                  'zillow': r_dict, 'source': 'zillow'})
    except Exception as exc:
        print('Unspecified error: {}'.format(exc))
        data_errors.append({'key': key, 'value': data_raw.pop(key),
                              'source': 'exception'})
            
    # log status
    print(i, ' ', zp['address'])

    endT = time.time()
    if endT - startT < 0.2 :
        time.sleep(0.2 - (endT-startT))     # rate limit to 5 calls per second

# save dictionaries back to disk
compressed = {}
compressed['data_raw'] = data_raw
compressed['data_queried'] = data_queried
compressed['data_errors'] = data_errors
with open(inProcessFile, 'wb') as fid :
    a = pickle.Pickler(fid)
    a.dump(compressed)

The most common error was code 508, 'no exact match found for input address'.  This was caused by the address not being in the Zillow database, such as the case for commercial buildings, churches, schools, etc., but could also be caused by an invalid address, such as a parcel without a street address ('None MacArthur Blvd').  The error rate was about 5%, or 1 in 20.  These results will be shown a little bit later on.

The final step to preparing the database is to write it back to a shapefile so it can be easily projected onto a choropleth map.  The schema from the original file is modified to include useful fields ('yearBuilt') and extra fields are discarded to speed up processing time.  Because this shapefile uses an ordered dictionary, the data fields need to be rearranged to match the schema's order.

In [ ]:
import fiona
import pickle
import os
from collections import OrderedDict
import numpy as np

inProcessFile = 'data/OaklandParcels_inProcess.pkl'         # data source
with open(inProcessFile, 'rb') as fid :
    compressed = pickle.load(fid)
# ...and unpack
data_raw = compressed['data_raw']
data_queried = compressed['data_queried']
del compressed

baseFile = 'data/Oakland_parcels/parcels'       # source of shape info in dictionary
outputFileName = 'Oakland_parcels_queried'
radius = 2000                                   # in m

# create output directory if it doesn't exist yet
if os.path.isdir('data/' + outputFileName) is False :
    os.makedirs('data/' + outputFileName)
outputFile = 'data/' + outputFileName + '/' + outputFileName + '.shp'

# Register format drivers with a context manager
with fiona.drivers():
    # get schema from original file
    with fiona.open(baseFile + '.shp') as source:
        meta = source.meta
        
    # add new fields to schema file
    meta['schema']['centroid'] = ('float:19:11', 'float:19:11')
    meta['schema']['id'] = 'float:19'
    meta['schema']['type'] = 'str:50'
    meta['schema']['yearBuilt'] = 'float:10'
    meta['schema']['properties']['YEARBUILT'] = 'int:6'
    meta['schema']['properties'] = OrderedDict(meta['schema']['properties'])
    schemaOrder = meta['schema']['properties']

    with fiona.open(outputFile, 'w', **meta) as sink:
        for (i,f) in enumerate(data_queried) :
            if f <= radius :   
                if 'yearBuilt' in data_queried[f]['zillow'] :
                    data_queried[f]['properties']['YEARBUILT'] = data_queried[f]['zillow']['yearBuilt']
                    data_queried[f].pop('zillow')
                else :
                    data_queried[f]['properties']['YEARBUILT'] = np.nan
                # reorder dictionary to match schema order
                data_queried[f]['properties'] = OrderedDict(
                        (k, data_queried[f]['properties'][k]) for k in schemaOrder)           
                sink.write(data_queried[f])

## Draw a Choropleth map

The best way to show the build year is graphically on a map, which will give an idea of which spatial patterns might require more analysis.  There is no straightforward implementation of this simple question, but there are multiple packages in python that can be strung together to draw a map, as well as a multitude of commercial software that does the same thing.

The general overview is to first choose a projection grid to map the round world onto.  This converts the (longitude, latitude) pairs to (x, y) pairs for the projection.  On top of this, the parcel polygons are drawn and colore